<a href="https://colab.research.google.com/github/gs1charancharan/Masai_assignments/blob/main/Responsible_AI_Practices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
df = pd.read_csv('diabetes_prediction_dataset.csv')

In [ ]:
df.head()


,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0


In [ ]:
!pip install -q pandas numpy matplotlib seaborn plotly scikit-learn fairlearn scipy

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                            f1_score, confusion_matrix, roc_auc_score)

# Fairness Libraries
from fairlearn.metrics import (
    demographic_parity_difference,
    demographic_parity_ratio,
    equalized_odds_difference,
    MetricFrame,
    selection_rate
)
from fairlearn.reductions import ExponentiatedGradient, DemographicParity

# Statistics
from scipy import stats

# Visualization settings
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("Set2")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 51.0 MB/s eta 0:00:00


In [ ]:
df = pd.read_csv('diabetes_prediction_dataset.csv')
print(df.head())

df_processed = df.copy()
df_processed['gender_encoded'] = (df_processed['gender'] == 'Male').astype(int)

smoking_map = {'never': 0, 'No Info': 1, 'not current': 2,
               'former': 3, 'ever': 4, 'current': 5}
df_processed['smoking_history_encoded'] = df_processed['smoking_history'].map(smoking_map)


df_processed['age_group'] = pd.cut(
    df_processed['age'],
    bins=[0, 30, 50, 65, 100],
    labels=['Young (0-30)', 'Middle (31-50)', 'Senior (51-65)', 'Elderly (65+)']
)

df_processed

   gender   age  hypertension  heart_disease smoking_history    bmi  \
0  Female  80.0             0              1           never  25.19   
1  Female  54.0             0              0         No Info  27.32   
2    Male  28.0             0              0           never  27.32   
3  Female  36.0             0              0         current  23.45   
4    Male  76.0             1              1         current  20.14   

   HbA1c_level  blood_glucose_level  diabetes  
0          6.6                  140         0  
1          6.6                   80         0  
2          5.7                  158         0  
3          5.0                  155         0  
4          4.8                  155         0  


,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes,gender_encoded,smoking_history_encoded,age_group
0,Female,80.0,0,1,never,25.19,6.6,140,0,0,0,Elderly (65+)
1,Female,54.0,0,0,No Info,27.32,6.6,80,0,0,1,Senior (51-65)
2,Male,28.0,0,0,never,27.32,5.7,158,0,1,0,Young (0-30)
3,Female,36.0,0,0,current,23.45,5.0,155,0,0,5,Middle (31-50)
4,Male,76.0,1,1,current,20.14,4.8,155,0,1,5,Elderly (65+)
...,...,...,...,...,...,...,...,...,...,...,...,...
99995,Female,80.0,0,0,No Info,27.32,6.2,90,0,0,1,Elderly (65+)
99996,Female,2.0,0,0,No Info,17.37,6.5,100,0,0,1,Young (0-30)
99997,Male,66.0,0,0,former,27.83,5.7,155,0,1,3,Elderly (65+)
99998,Female,24.0,0,0,never,35.42,4.0,100,0,0,0,Young (0-30)


In [ ]:
print("🤖 STEP 3: Training Baseline Model")

feature_cols = ['gender_encoded', 'age', 'hypertension', 'heart_disease',
                'smoking_history_encoded', 'bmi', 'HbA1c_level', 'blood_glucose_level']

X = df_processed[feature_cols]
y = df_processed['diabetes']

# Store sensitive features
sensitive_features = df_processed[['gender', 'age_group']]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

sensitive_train = sensitive_features.loc[X_train.index].reset_index(drop=True)
sensitive_test = sensitive_features.loc[X_test.index].reset_index(drop=True)

print(f"Training: {len(X_train):,} | Testing: {len(X_test):,}")

# Train model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=10)
rf_model.fit(X_train, y_train)

# Predictions
y_pred = rf_model.predict(X_test)
y_pred_proba = rf_model.predict_proba(X_test)[:, 1]

# Metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"\n📊 MODEL PERFORMANCE:")
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-Score:  {f1:.4f}")

🤖 STEP 3: Training Baseline Model
Training: 80,000 | Testing: 20,000

📊 MODEL PERFORMANCE:
Accuracy:  0.9723
Precision: 0.9991
Recall:    0.6747
F1-Score:  0.8055


In [ ]:
print("\n" + "="*80)
print("⚖️ STEP 4: Calculating Fairness Metrics")
print("="*80)

# Reset indices
y_test_reset = y_test.reset_index(drop=True)
y_pred_reset = pd.Series(y_pred).reset_index(drop=True)


# METRIC 1: Demographic Parity
print("\n1. DEMOGRAPHIC PARITY:")
dpd_gender = demographic_parity_difference(
    y_test_reset, y_pred_reset,
    sensitive_features=sensitive_test['gender']
)
dpr_gender = demographic_parity_ratio(
    y_test_reset, y_pred_reset,
    sensitive_features=sensitive_test['gender']
)

print(f"   Difference: {dpd_gender:.4f}")
print(f"   Ratio: {dpr_gender:.4f}")
print(f"   Status: {'✅ FAIR' if abs(dpd_gender) < 0.1 else '⚠️ BIAS DETECTED'}")


⚖️ STEP 4: Calculating Fairness Metrics

1. DEMOGRAPHIC PARITY:
   Difference: 0.0669
   Ratio: 0.0000
   Status: ✅ FAIR


In [ ]:
# METRIC 2: Equalized Odds
print("\n2. EQUALIZED ODDS:")
eod_gender = equalized_odds_difference(
    y_test_reset, y_pred_reset,
    sensitive_features=sensitive_test['gender']
)

print(f"   Difference: {eod_gender:.4f}")
print(f"   Status: {'✅ FAIR' if abs(eod_gender) < 0.1 else '⚠️ BIAS DETECTED'}")

# TPR and FPR by group
for gender in ['Female', 'Male']:
    mask = sensitive_test['gender'] == gender
    y_true_group = y_test_reset[mask]
    y_pred_group = y_pred_reset[mask]

    tpr = recall_score(y_true_group, y_pred_group, zero_division=0)
    tn, fp, fn, tp = confusion_matrix(y_true_group, y_pred_group).ravel()
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0

    print(f"   {gender}: TPR={tpr:.4f}, FPR={fpr:.4f}")


2. EQUALIZED ODDS:
   Difference: 0.6853
   Status: ⚠️ BIAS DETECTED
   Female: TPR=0.6652, FPR=0.0000
   Male: TPR=0.6853, FPR=0.0001


In [ ]:
# Prepare scaled data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train fair model
print("Training fairness-constrained model...")
baseline_estimator = LogisticRegression(random_state=42, max_iter=1000)

fair_classifier = ExponentiatedGradient(
    baseline_estimator,
    constraints=DemographicParity(),
    max_iter=50
)

fair_classifier.fit(
    X_train_scaled,
    y_train,
    sensitive_features=sensitive_train['gender']
)

print("✅ Fair model trained\n")

# Evaluate fair model
y_pred_fair = fair_classifier.predict(X_test_scaled)

fair_accuracy = accuracy_score(y_test_reset, y_pred_fair)
fair_dpd = demographic_parity_difference(
    y_test_reset, y_pred_fair,
    sensitive_features=sensitive_test['gender']
)
fair_eod = equalized_odds_difference(
    y_test_reset, y_pred_fair,
    sensitive_features=sensitive_test['gender']
)

# Comparison
print("📊 COMPARISON: Baseline vs Fair Model")
print("="*80)
print(f"{'Metric':<30} {'Baseline':<15} {'Fair Model':<15} {'Change'}")
print("-"*80)
print(f"{'Accuracy':<30} {accuracy:<15.4f} {fair_accuracy:<15.4f} {fair_accuracy-accuracy:+.4f}")
print(f"{'Demographic Parity Diff':<30} {dpd_gender:<15.4f} {fair_dpd:<15.4f} {fair_dpd-dpd_gender:+.4f}")
print(f"{'Equalized Odds Diff':<30} {eod_gender:<15.4f} {fair_eod:<15.4f} {fair_eod-eod_gender:+.4f}")


Training fairness-constrained model...
✅ Fair model trained

📊 COMPARISON: Baseline vs Fair Model
Metric                         Baseline        Fair Model      Change
--------------------------------------------------------------------------------
Accuracy                       0.9723          0.9405          -0.0319
Demographic Parity Diff        0.0669          0.4289          +0.3620
Equalized Odds Diff            0.6853          0.5945          -0.0908


In [ ]:
df_processed

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes,gender_encoded,smoking_history_encoded,age_group
0,Female,80.0,0,1,never,25.19,6.6,140,0,0,0,Elderly (65+)
1,Female,54.0,0,0,No Info,27.32,6.6,80,0,0,1,Senior (51-65)
2,Male,28.0,0,0,never,27.32,5.7,158,0,1,0,Young (0-30)
3,Female,36.0,0,0,current,23.45,5.0,155,0,0,5,Middle (31-50)
4,Male,76.0,1,1,current,20.14,4.8,155,0,1,5,Elderly (65+)
...,...,...,...,...,...,...,...,...,...,...,...,...
99995,Female,80.0,0,0,No Info,27.32,6.2,90,0,0,1,Elderly (65+)
99996,Female,2.0,0,0,No Info,17.37,6.5,100,0,0,1,Young (0-30)
99997,Male,66.0,0,0,former,27.83,5.7,155,0,1,3,Elderly (65+)
99998,Female,24.0,0,0,never,35.42,4.0,100,0,0,0,Young (0-30)


In [ ]:
def check_k_anonymity(df, quasi_ids, k=5):
    """Check if dataset satisfies k-anonymity"""
    df_check = df.copy()
    df_check['age_bin'] = pd.cut(df_check['age'], bins=10)

    groups = df_check.groupby(['gender', 'age_bin']).size().reset_index(name='count')
    vulnerable = groups[groups['count'] < k]

    return {
        'total_groups': len(groups),
        'vulnerable_groups': len(vulnerable),
        'vulnerable_records': vulnerable['count'].sum(),
        'k_anonymous': len(vulnerable) == 0,
        'min_group_size': groups['count'].min(),
        'max_group_size': groups['count'].max()
    }

# Test different k values
k_values = [2, 5, 10, 20]

print("K-ANONYMITY ANALYSIS:")
for k in k_values:
    result = check_k_anonymity(df_processed, ['gender', 'age'], k=k)
    print(f"\nk = {k}:")
    print(f"   Total groups: {result['total_groups']:,}")
    print(f"   Vulnerable groups: {result['vulnerable_groups']:,}")
    print(f"   Status: {'✅ k-Anonymous' if result['k_anonymous'] else '❌ Not k-Anonymous'}")


K-ANONYMITY ANALYSIS:

k = 2:
   Total groups: 30
   Vulnerable groups: 5
   Status: ❌ Not k-Anonymous

k = 5:
   Total groups: 30
   Vulnerable groups: 9
   Status: ❌ Not k-Anonymous

k = 10:
   Total groups: 30
   Vulnerable groups: 10
   Status: ❌ Not k-Anonymous

k = 20:
   Total groups: 30
   Vulnerable groups: 10
   Status: ❌ Not k-Anonymous
